In [2]:
import os
import torch
import wandb
from Data import *
import pandas as pd
from helper import *
from Models import *
import torch.nn as nn
import torch.optim as optim
from choosedataset import *
from Data_Embedding import *
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModel
from torch.nn.utils.rnn import pack_padded_sequence
import warnings
warnings.filterwarnings('ignore') # Ignore all warnings

# start

In [11]:
data = [Codeworkout, Falcon, Both][0]()
df = data.df

In [10]:
df.correct_eventually.value_counts()

correct_eventually
True     29757
False     6323
Name: count, dtype: int64

In [14]:
df.CorrectEventually.value_counts()

CorrectEventually
True     7591
False     379
Name: count, dtype: int64

In [8]:
text_model_name = 'google-bert/bert-base-uncased'
code_model_name = 'microsoft/codebert-base'

text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
code_tokenizer = AutoTokenizer.from_pretrained(code_model_name)
if text_tokenizer.pad_token is None:
    text_tokenizer.pad_token = text_tokenizer.eos_token
if code_tokenizer.pad_token is None:
    code_tokenizer.pad_token = code_tokenizer.eos_token
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
df['Label'].value_counts()
feature = ['IfElse', 'Loops', 'MathOperations', 'LogicOperators', 'StringOperations', 'List', 'FileOperations', 'Functions', 'Dictionary', 'Tuple']
# feature = ['IfElse', 'Loops', 'MathOperations', 'LogicOperators', 'StringOperations', 'List', 'FileOperations', 'Functions', 'Dictionary', 'Tuple', 'comprehension', 'alg_design', 'decomposition']


In [9]:
df.columns

Index(['student_id', 'assignment_id', 'problem_id', 'start_time', 'score',
       'max_score', 'question', 'prev_question', 'prev_code', 'prev_score',
       'prev_submission_score', 'prev_start_time', 'prev_label', 'Label',
       'Below75', 'CorrectEventually', 'IfElse', 'Loops', 'MathOperations',
       'LogicOperators', 'StringOperations', 'List', 'FileOperations',
       'Functions', 'Dictionary', 'Tuple', 'num_attempts', 'comprehension',
       'alg_design', 'decomposition', 'prev_comp_cons'],
      dtype='object')

In [10]:
# df['prev_code'] = df['prev_code'].apply(lambda x: [[i[-1]] for i in x]) # Only submission
df['prev_code'] = df['prev_code'].apply(lambda x: [i[-100:] for i in x])
# df['score'] = df['score'].apply(lambda x: [i[-100:] for i in x])

In [19]:
train_dataloader, valid_dataloader, test_dataloader = create_data_loader(df, Dataset_Embedding_Q, text_tokenizer, code_tokenizer, balanced_def=same_df, limit=30, padding_size_q=30, padding_size_code=100, batch_size=32, create_split=False, ids_filepath_prefix='Data/both/split_ids', feature=feature)

Load exist spliting


In [12]:
def num_of(train_dataloader, valid_dataloader, test_dataloader):
    print(len(train_dataloader), len(valid_dataloader), len(test_dataloader))
    print(len(set(train_dataloader.dataset.df['student_id'])), len(set(valid_dataloader.dataset.df['student_id'])), len(set(test_dataloader.dataset.df['student_id'])))
    print(set(train_dataloader.dataset.df['student_id']).intersection(set(valid_dataloader.dataset.df['student_id'])))
    print(set(train_dataloader.dataset.df['student_id']).intersection(set(test_dataloader.dataset.df['student_id'])))
    print(set(valid_dataloader.dataset.df['student_id']).intersection(set(test_dataloader.dataset.df['student_id'])))

In [13]:
num_of(train_dataloader, valid_dataloader, test_dataloader)

175 25 51
287 40 83
set()
set()
set()


In [ ]:
model = AllWithTaxonomyQmatrix(text_model_name)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [34]:
criterion = nn.BCEWithLogitsLoss() # pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [35]:
model = model.to(device)

In [ ]:
weights = [0.244, 0.756]
model = training_loop(model=model, train_dataloader=train_dataloader, test_dataloader=valid_dataloader, optimizer=optimizer, criterion=criterion, device=device, name="a", weights=None) # lr_scheduler=lr_schedule

In [ ]:
    # def custom_backward(self):
    #     with torch.no_grad(): # Step1: Mask the gradients to only update relevant KCs
    #         filtered_gradients = self.KCweight.weight.grad * self.mask
    #     with torch.no_grad(): # Step 2: Apply gradient descent step (standard update for weights)
    #         self.KCweight.weight -= filtered_gradients
    #     with torch.no_grad(): # Step 3: Apply positive weight constraint (ReLU to enforce positivity)
    #         self.KCweight.weight.data = torch.maximum(self.KCweight.weight.data, torch.zeros_like(self.KCweight.weight.data))
    #     with torch.no_grad(): # Step 4: Normalize the weights so they sum to 1 (per problem/task)
    #         weight_sums = self.KCweight.weight.sum()
    #         self.KCweight.weight.data = self.KCweight.weight.data / weight_sums    

In [16]:
path_models = "/home/nogaschw/Codeworkout/Thesis/Models"
files = os.listdir(path_models)
display(files)


['My full model (cw) q-matrix .pth',
 'My full model (cw) sum .pth',
 'My full model (cw) sum-norm .pth',
 'My full model (cw) (b) .pth',
 'My full model (cw) without taxonomy.pth',
 'Falcon',
 'My full model (cw) without taxonomy and past snapshots.pth',
 'My full model (cw) Prev (b) .pth',
 'My full model (falcon) PS without taxonomy.pth',
 'My full model (cw) Q-matrix smaller .pth',
 'My full model (cw) single Q-matrix smaller .pth',
 'My full model (cw) prev Q-matrix smaller .pth',
 'My full model (cw) Q-matrix skills .pth',
 'My full model (cw) q-matrix with backward .pth',
 'My full model (cw) with stat without taxonomy.pth',
 'My full model (cw) with improves without taxonomy and past snapshots.pth',
 'My full model (cw) with lrwithout taxonomy.pth',
 'My full model (cw) imbalancewithout taxonomy.pth',
 'My full model (cw) attention without taxonomy.pth',
 'My full model (falcon) Pearson similarity without taxonomy.pth',
 'My full model (cw) stats as in comp without taxonomy.pth

In [23]:
model_path_name = os.path.join(path_models, f'My full model (cw) 32 batches without taxonomy.pth')
print(model_path_name)
model.load_state_dict(torch.load(model_path_name))
model.to(device)

/home/nogaschw/Codeworkout/Thesis/Models/My full model (cw) 32 batches without taxonomy.pth


All(
  (text_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
wandb.login()

wandb.init(
    project="Struggling Students",  # Set your project name
    entity="naji1",                 # Your WandB username or team
    id="v0dhrg3n",                  # The run ID of the run you want to resume
    resume="allow",                 # This allows WandB to resume the run
)

In [24]:
from sklearn.metrics import roc_curve
 
all_labels, all_probs = eval_loop(model, valid_dataloader, device)

fpr, tpr, thresholds = roc_curve(all_labels, all_probs)

# J = tpr - fpr
# best_index = J.argmax()
# best_threshold = thresholds[best_index]

Batch 0 from 25


In [26]:
J = tpr - fpr
best_index = J.argmax()
thresholds[best_index]

np.float64(0.4545215666294098)

In [27]:
y_labels, y_probs = eval_loop(model, test_dataloader, device)

Batch 0 from 51


In [34]:
roc_auc = roc_auc_score(y_labels, y_probs)
roc_auc

np.float64(0.6915131942031356)

In [28]:
from sklearn.metrics import f1_score, precision_score, matthews_corrcoef

def find_f1_threshold(all_labels, all_probs, thresholds):
    """Finds the threshold that maximizes F1 Score."""
    best_threshold = thresholds[0]
    best_f1 = 0
    for threshold in thresholds:
        predictions = (all_probs >= threshold).astype(int)
        f1 = f1_score(all_labels, predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

def find_geometric_mean_threshold(fpr, tpr, thresholds):
    """Finds the threshold that maximizes the Geometric Mean of TPR and TNR (1 - FPR)."""
    best_threshold = thresholds[0]
    best_geometric_mean = 0
    for idx, threshold in enumerate(thresholds):
        geometric_mean = np.sqrt(tpr[idx] * (1 - fpr[idx]))
        if geometric_mean > best_geometric_mean:
            best_geometric_mean = geometric_mean
            best_threshold = threshold
    return best_threshold

def find_mcc_threshold(all_labels, all_probs, thresholds):
    """Finds the threshold that maximizes Matthews Correlation Coefficient (MCC)."""
    best_threshold = thresholds[0]
    best_mcc = -1  # MCC ranges from -1 to +1, so start lower than the minimum possible value
    for threshold in thresholds:
        predictions = (all_probs >= threshold).astype(int)
        mcc = matthews_corrcoef(all_labels, predictions)
        if mcc > best_mcc:
            best_mcc = mcc
            best_threshold = threshold
    return best_threshold

def find_precision_threshold(all_labels, all_probs, thresholds):
    """Finds the threshold that maximizes Precision."""
    best_threshold = thresholds[0]
    best_precision = 0
    for threshold in thresholds:
        predictions = (all_probs >= threshold).astype(int)
        precision = precision_score(all_labels, predictions)
        if precision > best_precision:
            best_precision = precision
            best_threshold = threshold
    return best_threshold


In [29]:
f1_score_threshold = find_f1_threshold(all_labels, all_probs, thresholds)
geometric_mean_threshold = find_geometric_mean_threshold(fpr, tpr, thresholds)
mcc_threshold = find_mcc_threshold(all_labels, all_probs, thresholds)
precision_threshold = find_precision_threshold(all_labels, all_probs, thresholds)

In [30]:
thershold_list = [f1_score_threshold, geometric_mean_threshold, mcc_threshold, precision_threshold, thresholds[best_index], 0.5]

In [31]:
def which_cm(pred, label):
    if pred == 1 and label == 0:
        return "FP"
    if pred == 0 and label == 0:
        return "TN"
    if pred == 1 and label == 1:
        return "TP"
    return "FN"



In [32]:
for i in thershold_list:
    y_pred = np.where(y_probs > np.float64(i), 1, 0)
    cm = []
    for j, (pred, label) in enumerate(zip(y_pred, y_labels)):
        if test_dataloader.dataset.df.iloc[j]['Label'] != label:
            print('problem', j)
            break
        cm.append(which_cm(pred, label))
    work = test_dataloader.dataset.df.copy()
    work["CM"] = cm
    print("thershold: ", i)
    print(work.CM.value_counts())
    print(recall_score(y_labels, y_pred), precision_score(y_labels, y_pred), f1_score(y_labels, y_pred), "\n")

thershold:  0.18481391668319702
CM
TP    1204
FP     406
FN       1
Name: count, dtype: int64
0.9991701244813278 0.7478260869565218 0.8554174067495559 

thershold:  0.42778509855270386
CM
TP    693
FN    512
TN    272
FP    134
Name: count, dtype: int64
0.575103734439834 0.837968561064087 0.6820866141732284 

thershold:  0.4545215666294098
CM
TP    618
FN    587
TN    302
FP    104
Name: count, dtype: int64
0.512863070539419 0.8559556786703602 0.6414115204981837 

thershold:  0.7726464867591858
CM
FN    1205
TN     406
Name: count, dtype: int64
0.0 0.0 0.0 

thershold:  0.4545215666294098
CM
TP    618
FN    587
TN    302
FP    104
Name: count, dtype: int64
0.512863070539419 0.8559556786703602 0.6414115204981837 

thershold:  0.5
CM
FN    735
TP    470
TN    342
FP     64
Name: count, dtype: int64
0.3900414937759336 0.8801498127340824 0.5405405405405406 



In [73]:
work.CM.value_counts()

CM
TP    828
FN    377
TN    271
FP    135
Name: count, dtype: int64

I told he is success but he struggling

In [27]:
work.columns

Index(['student_id', 'assignment_id', 'problem_id', 'start_time', 'score',
       'max_score', 'question', 'prev_question', 'prev_code', 'prev_score',
       'prev_submission_score', 'prev_start_time', 'prev_label', 'Label',
       'Below75', 'CorrectEventually', 'IfElse', 'Loops', 'MathOperations',
       'LogicOperators', 'StringOperations', 'List', 'FileOperations',
       'Functions', 'Dictionary', 'Tuple', 'comprehension', 'alg_design',
       'decomposition', 'num_attempts', 'CM'],
      dtype='object')

In [50]:
def filter_by_bool_array(row):
    # Use numpy boolean indexing to filter `True` positions in `bool_array
    bool_indices = pd.Series(row['prev_label'])
    
    # Filter `values` and `labels` based on `True` values in `bool_array`
    row['prev_code'] = [v for v, b in zip(row['prev_code'], bool_indices) if b]
    row['prev_question'] = [l for l, b in zip(row['prev_question'], bool_indices) if b]
    return row

In [75]:
len(work[work.CM == 'FP'])

135

In [74]:
work[work.CM == 'FP'].problem_id.value_counts()


problem_id
71     13
43     13
44     12
67     12
49      8
64      7
51      7
70      7
118     7
57      6
106     6
48      6
104     5
41      5
56      5
46      4
112     4
45      4
107     3
108     1
Name: count, dtype: int64

In [127]:
len(set(df.student_id.values))

410

In [131]:
df.columns

Index(['student_id', 'assignment_id', 'problem_id', 'start_time', 'score',
       'max_score', 'question', 'prev_question', 'prev_code', 'prev_score',
       'prev_submission_score', 'prev_start_time', 'prev_label', 'Label',
       'Below75', 'CorrectEventually', 'IfElse', 'Loops', 'MathOperations',
       'LogicOperators', 'StringOperations', 'List', 'FileOperations',
       'Functions', 'Dictionary', 'Tuple', 'comprehension', 'alg_design',
       'decomposition'],
      dtype='object')

In [133]:
# Temporarily convert lists to tuples in `prev_code` column (or any other column with lists)
df_big['prev_code_tuple'] = df_big['prev_code'].apply(tuple)

# Now drop duplicates based on the column with tuples
df_big = df_big.drop_duplicates(subset=['prev_code_tuple'])

# Optionally, remove the tuple column if it's no longer needed
df_big.drop(columns=['prev_code_tuple'], inplace=True)

In [120]:
tuple(df_big['prev_code'].iloc[10000])

([['public int sortaSum(int a, int b)\r\n{\r\n  int c = a+b;\r\n  if( c>9 && c<20)\r\n  {\r\n      return 20;\r\n  }\r\n  else\r\n  {\r\n      return c;\r\n  }\r\n    \r\n}\r\n']],
 [['public boolean in1To10(int n, boolean outsideMode)\r\n{\r\n    if( n>0 && n<11 && !(outsideMode))\r\n    {\r\n        return true;\r\n    }\r\n    else if(outsideMode && (n<= 1 || n>=10))\r\n    {\r\n        return true;\r\n    }\r\n    else \r\n    {\r\n        return false;\r\n    }\r\n    \r\n}\r\n']],
 [['public boolean answerCell(boolean isMorning, boolean isMom, boolean isAsleep)\r\n{\r\n   if( isMorning && isMom)\r\n   {\r\n       return true;\r\n   }\r\n   else if ( isMorning && !(isMom))\r\n   {\r\n       return false;\r\n   }\r\n   else if ( isAsleep)\r\n   {\r\n       return false;\r\n   }\r\n   else\r\n   {\r\n       return true;\r\n   }\r\n}\r\n']],
 [['public boolean squirrelPlay(int temp, boolean isSummer)\r\n{\r\n    int tempUp = 0;\r\n    if(isSummer)\r\n    {\r\n        tempUp = temp -1

In [95]:
df['prev_code'].apply(lambda x: max([len(i) for i in x]))

0       38
1       38
2       38
3       38
4       38
        ..
7965    10
7966    10
7967    10
7968    10
7969    10
Name: prev_code, Length: 7970, dtype: int64

In [87]:
work[work.CM == 'FP'].problem_id.value_counts()

problem_id
44     13
71     13
67     12
43     11
49      8
64      7
118     7
70      7
48      6
57      6
51      6
56      5
41      5
104     4
46      4
45      4
106     4
107     3
112     3
108     1
Name: count, dtype: int64

In [85]:
work[work.CM == 'FP'].Below75.value_counts()

Below75
False    108
True      21
Name: count, dtype: int64

In [54]:
a = work[(work.CM == 'FP') & (work.problem_id == 67)].apply(filter_by_bool_array, axis=1).prev_question

In [58]:
from collections import Counter

flat_list = [item for sublist in list(a) for item in sublist]
count_dict = dict(Counter(flat_list))

In [56]:
work[work.student_id == '321ceeadc24ba2a88fba401927295f3c972429fcf3873d45a5cd9134279987cb'].Label.value_counts()

Label
False    14
True      6
Name: count, dtype: int64

In [41]:
work[(work.CM == 'FP') & (work.problem_id == 67)].iloc[0]['prev_question'][14]

'Write a function in Java that implements the following logic: Given 2 int values greater than 0, return whichever value is nearest to 21 without going over. Return 0 if they both go over.'

In [44]:
work[(work.CM == 'FP') & (work.problem_id == 67)]

,student_id,assignment_id,problem_id,start_time,score,max_score,question,prev_question,prev_code,prev_score,...,StringOperations,List,FileOperations,Functions,Dictionary,Tuple,comprehension,alg_design,decomposition,CM
1480,321ceeadc24ba2a88fba401927295f3c972429fcf3873d...,494,67,2019-11-09 02:23:57+00:00,"[0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.55, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.0, 1.0], [1.0], [1.0], [1.0], [0.0, 0.0, 0...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
2189,48f565383f47c6752893810686bd0c5f07fef08e34bc22...,494,67,2019-11-10 20:21:29+00:00,"[0.9, 0.9, 0.9, 0.0, 0.9, 0.9, 0.9, 0.9, 0.0, ...",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.6, 0.6, 0.0, 0.0, 0.2, 0.2, 0.2, 0.466667,...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
3237,6b689d246c11ea35bf66a151aa4d7d83d1e1b11f9f3a38...,494,67,2019-11-12 03:55:28+00:00,"[0.0, 0.0, 0.0, 0.35, 0.9, 0.9, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n\t...","[[0.466667, 0.466667, 1.0], [0.0, 0.0, 0.25, 0...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
3470,753b3dfbf00b265816877af61db67c39ee3fae2b4e0a7b...,494,67,2019-11-10 23:46:43+00:00,"[0.9, 0.9, 0.8, 0.0, 0.0, 0.0, 0.9, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.466667, 1.0], [1.0], [1.0], [0.0, 1.0], [0...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
3749,7a6026fd215e0059f3d620352b64b05a944244be51bbf2...,494,67,2019-11-11 14:35:45+00:00,"[0.7, 0.9, 0.8, 0.9, 0.7, 0.9, 0.9, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.733333, 1.0], [0.0, 1.0], [1.0], [1.0], [0...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
4164,8b18f5693d6822beefe70802cad008f0b0232654ea218f...,494,67,2019-11-08 01:45:13+00:00,"[0.7, 0.65, 0.45, 0.45, 0.9, 0.9, 0.9, 0.45, 0...",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.733333, 0.0, 1.0], [0.375, 0.4375, 0.0, 0....",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
4638,98d77a2b714b28f705c31ae807ec9dc0e2e5415b10f7bd...,494,67,2019-11-12 03:07:59+00:00,"[0.0, 0.7, 0.9, 0.7, 0.0, 0.8, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.466667, 0.0, 0.466667, 1.0], [0.0, 0.0, 0....",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
4908,a1a4b0c316442b879083c61343bcb56ce2233230e64fc4...,494,67,2019-11-11 20:41:28+00:00,"[0.55, 0.0, 0.0, 0.0, 0.55, 0.0, 0.5, 0.0, 0.0...",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.333333, 0.466667, 1.0], [0.0, 0.0, 0.0, 0....",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
4928,a1a57315af0421443e73ff39f4cc4afaabfb05e5c435fa...,494,67,2019-11-10 22:28:40+00:00,"[0.0, 0.55, 0.75, 0.9, 0.9, 0.0, 0.0, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.6875,...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP
6085,c8d01afd1e4a742fb48896441f92f6f4f1c21f02b0059b...,494,67,2019-11-08 00:22:32+00:00,"[0.0, 0.5, 0.65, 0.0, 0.9, 0.0, 1.0]",100.0,"Given an array of ints, return true if the arr...",[Write a function in Java that implements the ...,"[[public int sortaSum(int a, int b)\r\n{\r\n ...","[[1.0], [0.0, 1.0], [1.0], [1.0], [0.0, 0.7826...",...,0.0,1.0,0,0.0,0,0,4,3,4,FP


In [59]:
work[(work.CM == 'FP') & (work.problem_id == 67)].prev_code.iloc[0]

[['public int sortaSum(int a, int b)\r\n{\r\n    if ( a + b >= 10 && a + b <=19 )\r\n    {\r\n        return 20;\r\n    }\r\n    else\r\n    {\r\n        return a + b;\r\n    }\r\n}\r\n',
  'public int sortaSum(int a, int b)\r\n{\r\n    if ( a + b >= 10 && <=19 )\r\n    {\r\n        return 20;\r\n    }\r\n    else\r\n    {\r\n        return a + b;\r\n    }\r\n}\r\n'],
 ['public boolean in1To10(int n, boolean outsideMode)\r\n{\r\n    if (outsideMode)\r\n    {\r\n        if (n <= 1 || n >=10)\r\n        {\r\n            return true;\r\n        }\r\n        else\r\n        {\r\n            return false;\r\n        }\r\n\t}\t\r\n    else if (n >= 1 && n <= 10)\r\n    {\r\n        return true;\r\n    }\r\n    else\r\n    {\r\n        return false;\r\n    }\r\n        \r\n}\r\n'],
 ['public boolean answerCell(boolean isMorning, boolean isMom, boolean isAsleep)\r\n{\r\n    if (isAsleep)\r\n    {\r\n        return false;\r\n    }\r\n    else\r\n    {\r\n        if (isMorning && isMom)\r\n     

In [28]:
work.columns

Index(['student_id', 'assignment_id', 'problem_id', 'start_time', 'score',
       'max_score', 'question', 'prev_question', 'prev_code', 'prev_score',
       'prev_submission_score', 'prev_start_time', 'prev_label', 'Label',
       'Below75', 'CorrectEventually', 'IfElse', 'Loops', 'MathOperations',
       'LogicOperators', 'StringOperations', 'List', 'FileOperations',
       'Functions', 'Dictionary', 'Tuple', 'comprehension', 'alg_design',
       'decomposition', 'CM'],
      dtype='object')

In [33]:
work[work.problem_id == 57].CorrectEventually.value_counts()

CorrectEventually
True    80
Name: count, dtype: int64

In [42]:
work[(work.problem_id == 57) & (work.Below75 != False) & (work.CM == 'FN')].student_id.iloc[0]

'125ae2e6409e498cf96017e4f7168a1612ea2f181a0ea99530cfafb7b3a64807'

In [99]:
work[(work.problem_id == 57) & (work.Below75 != False) & (work.CM == 'FN')].prev_label.apply(sum)

494     15
693     13
944     25
1189    25
2078    19
2755    25
2775    27
2795    15
3954    25
5017    19
5587    21
5734    26
5974    16
6269    18
6483    20
6654    20
6694    14
6871    20
7212    22
7824    22
Name: prev_label, dtype: int64

In [108]:
work[work.CM == 'FN'].prev_label.apply(sum).value_counts()

prev_label
25    73
19    39
21    34
22    33
20    33
26    32
15    30
23    21
14    14
24    13
3     12
17    11
27    11
16    11
18    10
13     8
29     4
5      1
Name: count, dtype: int64

In [112]:
work[work.CM == 'FP'].prev_label.apply(sum).value_counts()

prev_label
28    23
21    22
22    18
20    13
19    12
25    10
24    10
27     9
26     5
15     2
29     2
30     2
5      2
17     1
Name: count, dtype: int64

In [64]:
work[(work.student_id == '125ae2e6409e498cf96017e4f7168a1612ea2f181a0ea99530cfafb7b3a64807') & ((work.CM == 'TN'))].question.iloc[0]

"Given an array, return an array that contains exactly the same numbers as the original array, but rearranged so that every 4 is immediately followed by a 5. Do not move the 4's, but every other number may move. The array contains the same number of 4's and 5's, and every 4 has a number after it that is not a 4. In this version, 5's may appear anywhere in the original array. (This is a slightly harder version of the fix34 problem.)"

In [100]:
work[work.problem_id == 104].Label.value_counts()

Label
True     58
False    22
Name: count, dtype: int64

In [174]:
work[(work.student_id == '125ae2e6409e498cf96017e4f7168a1612ea2f181a0ea99530cfafb7b3a64807') & (work.CM == 'FN')].question.iloc[0]

"Given an array, return an array that contains exactly the same numbers as the original array, but rearranged so that every 4 is immediately followed by a 5. Do not move the 4's, but every other number may move. The array contains the same number of 4's and 5's, and every 4 has a number after it that is not a 4. In this version, 5's may appear anywhere in the original array. (This is a slightly harder version of the fix34 problem.)"

In [109]:
print(work[(work.CM == 'FP') & (work.Below75 == False)].prev_question.iloc[0][2])

Write a function in Java that implements the following logic: Your cell phone rings. Return true if you should answer it. Normally you answer, except in the morning you only answer if it is your mom calling. In all cases, if you are asleep, you do not answer.


In [160]:
False, 'Write a function in Java that implements the following logic: Your cell phone rings. Return true if you should answer it. Normally you answer, except in the morning you only answer if it is your mom calling. In all cases, if you are asleep, you do not answer.'

True, 'FN', "Given an array, return an array that contains exactly the same numbers as the original array, but rearranged so that every 4 is immediately followed by a 5. Do not move the 4's, but every other number may move. The array contains the same number of 4's and 5's, and every 4 has a number after it that is not a 4. In this version, 5's may appear anywhere in the original array. (This is a slightly harder version of the fix34 problem.)"

False, 'FP', 'Write a function in Java that implements the following logic: Return a version of the given array where each zero value in the array is replaced by the largest odd value to the right of the zero in the array. If there is no odd value to the right of the zero, leave the zero as a zero. Hint: in solving this: you may use a second helper function if you want, say to find the largest odd value to the right of a specified position.'

True, 'TP', 'Given an array containing three ints, return the sum of all the elements.'

False, 'TN', 'Consider the series of numbers beginning at start and running up to but not including end, so for example start=1 and end=5 gives the series 1, 2, 3, 4. Return a new String[] array containing the string form of these numbers, except in the case of multiples of 3, use "Fizz" instead of the number, and for multiples of 5 use "Buzz", and for multiples of both 3 and 5 use "FizzBuzz". In Java, String.valueOf(xxx) will make the String form of an int or other type.'

False, 'TN', 'Given n >= 0, create an array with the pattern {1, 1, 2, 1, 2, 3, ... 1, 2, 3 .. n} (spaces added to show the grouping). Note that the length of the array will be 1 + 2 + 3 ... + n, which is known to sum to exactly n * (n + 1)/2.'

False, 'FP', 'Given an array of int values, return true if the group of n numbers at the start of the array is the same as the group of n numbers at the end of the array. For example, with {5, 6, 45, 99, 13, 5, 6}, the ends are the same for n == 0 and n == 2, and false for n == 1 and n == 3. You may assume that n is in the range 0..nums.length, inclusive.'

False, 'FP', "Given an int[] array, write a function that returns an array where all the 10's have been removed. The remaining elements should shift left towards the start of the array as needed, and the empty spaces at the end of the array should be set to 0. So {1, 10, 10, 2} yields {1, 2, 0, 0}. You may modify and return the given array or make a new array."

True, 'FN', 'Given a number n, create and return a new int array of length n, containing the numbers 0, 1, 2, ... n-1. The given n may be 0, in which case just return a length 0 array. You do not need a separate if statement for the length-0 case; the for loop should naturally execute 0 times in that case, so it just works. The syntax to make a new int array is new int[desired_length].'

False, 'Write a function in Java that implements the following logic: The squirrels in Palo Alto spend most of the day playing. In particular, they play if the temperature is between 60 and 90 (inclusive). Unless it is summer, then the upper limit is 100 instead of 90. Given an int temperature and a boolean isSummer, return true if the squirrels play and false otherwise.'


'Given a number n, create and return a new int array of length n, containing the numbers 0, 1, 2, ... n-1. The given n may be 0, in which case just return a length 0 array. You do not need a separate if statement for the length-0 case; the for loop should naturally execute 0 times in that case, so it just works. The syntax to make a new int array is new int[desired_length].'

In [91]:
work[(work.CM == 'FP') & (work.Below75 == False)]['prev_code'].iloc[5]

[['public int sortaSum(int a, int b)\r\n{\r\n    if ( a + b >= 10 && a + b <=19 )\r\n    {\r\n        return 20;\r\n    }\r\n    else\r\n    {\r\n        return a + b;\r\n    }\r\n}\r\n',
  'public int sortaSum(int a, int b)\r\n{\r\n    if ( a + b >= 10 && <=19 )\r\n    {\r\n        return 20;\r\n    }\r\n    else\r\n    {\r\n        return a + b;\r\n    }\r\n}\r\n'],
 ['public boolean in1To10(int n, boolean outsideMode)\r\n{\r\n    if (outsideMode)\r\n    {\r\n        if (n <= 1 || n >=10)\r\n        {\r\n            return true;\r\n        }\r\n        else\r\n        {\r\n            return false;\r\n        }\r\n\t}\t\r\n    else if (n >= 1 && n <= 10)\r\n    {\r\n        return true;\r\n    }\r\n    else\r\n    {\r\n        return false;\r\n    }\r\n        \r\n}\r\n'],
 ['public boolean answerCell(boolean isMorning, boolean isMom, boolean isAsleep)\r\n{\r\n    if (isAsleep)\r\n    {\r\n        return false;\r\n    }\r\n    else\r\n    {\r\n        if (isMorning && isMom)\r\n     

In [101]:
a = work[work['problem_id'].map(work['problem_id'].value_counts()) > 30]

In [102]:
a = (a[a.CM == 'FP'].problem_id.value_counts() / a.problem_id.value_counts()).fillna(0)

In [134]:
work[work.problem_id == 'lsn9_skill3']['question'].iloc[0]

" Define a function called print_hello() . Your function should, when called, simply print out a message of your choosing that would be a greeting to someone. IMPORTANT NOTE: Functions only run when a program calls them. For the purposes of this exercise, you do not need to call the function. If you don't, however, nothing will appear on the screen when you run it in Thonny."

In [109]:
a.sort_values()

problem_id
lsn11_skill1             0.000000
lsn13_skill2             0.000000
lsn19_skill2             0.000000
lsn15_titanicsurvivor    0.000000
lsn16_practice2          0.017544
                           ...   
lsn9_skill3              0.242424
lsn15_skill2             0.243697
lsn11_skill3             0.255814
lsn13_averagerun         0.287234
lists1_listrange         0.295082
Name: count, Length: 64, dtype: float64

In [ ]:
feature = ['IfElse', 'Loops', 'MathOperations', 'LogicOperators', 'StringOperations', 'List', 'FileOperations', 'Functions', 'Dictionary', 'Tuple']


In [63]:
work[work.Functions == True].CM.value_counts().head(20)

CM
TP    1138
TN     951
FN     749
FP     354
Name: count, dtype: int64

In [61]:
work[work.CM == 'FP'].FileOperations.value_counts().head(20)

FileOperations
0    537
1    122
Name: count, dtype: int64

In [18]:
cm = confusion_matrix(y_labels, y_pred)
print(cm)
roc_auc = roc_auc_score(y_labels, y_probs)
accuracy = accuracy_score(y_labels, y_pred)
precision = precision_score(y_labels, y_pred)
recall = recall_score(y_labels, y_pred)
f1 = f1_score(y_labels, y_pred)
roc_auc, accuracy, precision, recall, f1

[[284 122]
 [584 621]]


(np.float64(0.6379668049792531),
 0.5617628801986344,
 np.float64(0.8358008075370121),
 np.float64(0.5153526970954356),
 np.float64(0.6375770020533881))

In [28]:
1945 / 6025

0.3228215767634855

In [46]:
cm = confusion_matrix(y_labels, y_pred)
print(cm)
roc_auc = roc_auc_score(y_labels, y_probs)
accuracy = accuracy_score(y_labels, y_pred)
precision = precision_score(y_labels, y_pred)
recall = recall_score(y_labels, y_pred)
f1 = f1_score(y_labels, y_pred)
roc_auc, accuracy, precision, recall, f1

[[1643  952]
 [2053 2592]]


(np.float64(0.6493060057948652),
 0.5849447513812155,
 np.float64(0.7313769751693002),
 np.float64(0.5580193756727664),
 np.float64(0.6330443277567469))

In [32]:
cm = confusion_matrix(y_labels, y_pred)
print(cm)
roc_auc = roc_auc_score(y_labels, y_probs)
accuracy = accuracy_score(y_labels, y_pred)
precision = precision_score(y_labels, y_pred)
recall = recall_score(y_labels, y_pred)
f1 = f1_score(y_labels, y_pred)
roc_auc, accuracy, precision, recall, f1

[[242 164]
 [379 826]]


(np.float64(0.6913108353943952),
 0.6629422718808193,
 np.float64(0.8343434343434344),
 np.float64(0.6854771784232365),
 np.float64(0.752619589977221))

In [20]:
wandb.finish()

accuracy,▁█
f1_0.5,▁
f1_best,▁
precision_0.5,▁
precision_best,▁
recall_0.5,▁
recall_best,▁
roc_auc,▁▁
threshold,█▁
accuracy,0.74798
avg_loss_valid loss,0.58279


# Models

### Not

#### Curr Q

In [20]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.layer1 = nn.Linear(self.text_model.config.hidden_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, num_snapshots, que_num): 
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        x = self.layer1(text_output)
        x = self.relu(x)
        x = self.layer2(x)
        return x

#### Prev Snapshots

In [9]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size, 1)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, num_snapshots, que_num): 
        packed_sequence = pack_padded_sequence(code_embedding.float(), lengths=code_num.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (h_n, c_n) = self.lstm(packed_sequence)
        output = self.layer1(h_n[-1])
        return output

#### Prev Snapshots + Curr Q

In [47]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size, 1)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, num_snapshots, que_num): 
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
            prev_q_output = self.text_model(questions_input_ids, questions_attention_mask).last_hidden_state[:, 0, :].unsqueeze(1)
        batch_indices = torch.arange(code_embedding.size(0), device=code_embedding.device)
        input_to_lstm =  torch.cat((prev_q_output, code_embedding[:, :(max(code_num) + 1), :]), dim=1).float()
        input_to_lstm[batch_indices, code_num + 1, :] = text_output
        packed_sequence = pack_padded_sequence(input_to_lstm.float(), lengths=code_num.cpu() + 2, batch_first=True, enforce_sorted=False)
        packed_output, (h_n, c_n) = self.lstm(packed_sequence)
        output = self.layer1(h_n[-1])
        return output

#### X Last snapshot + Curr Q

In [12]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size, 1)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        input_to_lstm = torch.cat((code_embedding, text_output.unsqueeze(1)), dim=1).float()
        packed_output, (h_n, c_n) = self.lstm(input_to_lstm)
        output = self.layer1(h_n[-1])
        return output

#### X Last snapshot & Binary 

before LSTM

In [36]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size + 1, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size, 1)
        self.batch_norm = nn.BatchNorm1d(769)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        batch_size = code_embedding.size(0)
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        last_s_curr_q = torch.cat((code_embedding, text_output.unsqueeze(1)), dim=1).float()
        binary_tensor = torch.cat([prev_struggling.float(), torch.zeros((batch_size, 1), dtype=torch.float, device=code_embedding.device)], dim=1).unsqueeze(2)
        unnormalized_input = torch.cat((last_s_curr_q, binary_tensor), dim=2).float()
        input_to_lstm = self.batch_norm(unnormalized_input.permute(0, 2, 1)).permute(0, 2, 1) 
        packed_output, (h_n, c_n) = self.lstm(input_to_lstm)
        output = self.layer1(h_n[-1])
        return output

After LSTM

In [ ]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size + question_num, 1)
        self.batch_norm = nn.BatchNorm1d(768)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        input_to_lstm = torch.cat((code_embedding, text_output.unsqueeze(1)), dim=1).float()
        packed_output, (h_n, c_n) = self.lstm(input_to_lstm)
        prev_struggling = prev_struggling.float()
        concatenated = torch.cat((h_n[-1], prev_struggling), dim=1)
        normalized = self.batch_norm(concatenated)
        output = self.layer1(normalized)
        return output

#### X Prev Q & Last snapshot + Curr Q 

In [19]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size, 1)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, num_snapshots, que_num): 
        batch_size, num_q , len = questions_input_ids.size()
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
            prev_q_output = self.text_model(
                questions_input_ids.view(-1, len), 
                questions_attention_mask.view(-1, len)
            ).last_hidden_state[:, 0, :].view(batch_size, num_q, -1)
        embedding = torch.empty((batch_size,  2 * num_q, code_embedding.size(-1)), dtype=torch.float, device=code_embedding.device)
        embedding[:, 0::2, :] = prev_q_output
        embedding[:, 1::2, :] = code_embedding
        embedding = torch.cat((embedding, text_output.unsqueeze(1)), dim=1).float()
        packed_output, (h_n, c_n) = self.lstm(embedding)
        output = self.layer1(h_n[-1])
        return output

In [20]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.layer1 = nn.Linear(hidden_size, 1)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, num_snapshots, que_num): 
        batch_size, num_q , len = questions_input_ids.size()
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
            prev_q_output = prev_questions_embedding.last_hidden_state[:, 0, :].unsqueeze(0).repeat(batch_size, 1, 1)
        embedding = torch.empty((batch_size,  2 * num_q, code_embedding.size(-1)), dtype=torch.float, device=code_embedding.device)
        embedding[:, 0::2, :] = prev_q_output
        embedding[:, 1::2, :] = code_embedding
        embedding = torch.cat((embedding, text_output.unsqueeze(1)), dim=1).float()
        packed_output, (h_n, c_n) = self.lstm(embedding)
        output = self.layer1(h_n[-1])
        return output

#### X Prev Q & Last snapshot & Binary struggling + Curr Q 

In [17]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(self.text_model.config.hidden_size, hidden_size, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_size + question_num)
        self.layer1 = nn.Linear(hidden_size + question_num, 1)

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        batch_size, num_q , len = questions_input_ids.size()
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
            prev_q_output = prev_questions_embedding.last_hidden_state[:, 0, :].unsqueeze(0).repeat(batch_size, 1, 1)
        embedding = torch.empty((batch_size, num_q * 2, code_embedding.size(-1)), dtype=torch.float, device=code_embedding.device)
        embedding[:, 0::2, :] = prev_q_output
        embedding[:, 1::2, :] = code_embedding
        embedding = torch.cat((embedding, text_output.unsqueeze(1)), dim=1).float()
        packed_output, (h_n, c_n) = self.lstm(embedding)
        prev_struggling = prev_struggling.float()
        concatenated = torch.cat((h_n[-1], prev_struggling), dim=1)
        normalized = self.batch_norm(concatenated)
        output = self.layer1(normalized)
        return output

### MLP

##### Smaller Layer

In [ ]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size_lstm=128 , hidden_dim_mlp=512, output_dim_mlp=256, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.MLP_text = nn.Sequential(
            nn.Linear(768, hidden_dim_mlp),
            nn.LayerNorm(hidden_dim_mlp),
            nn.ReLU(),
            nn.Linear(hidden_dim_mlp, output_dim_mlp),
            nn.LayerNorm(output_dim_mlp)
        )
        self.MLP_code = nn.Sequential(
            nn.Linear(768, hidden_dim_mlp),
            nn.LayerNorm(hidden_dim_mlp),
            nn.ReLU(),
            nn.Linear(hidden_dim_mlp, output_dim_mlp),
            nn.LayerNorm(output_dim_mlp)
        )
        self.lstm = nn.LSTM(output_dim_mlp, hidden_size_lstm, batch_first=True)
        self.layer1 = nn.Linear(hidden_size_lstm, 1)
        
    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        batch_size, num_q, _ = code_embedding.size()
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        mlp_text_output = self.MLP_text(text_output) 
        mlp_code_output = self.MLP_code(code_embedding.view(batch_size * num_q, -1)).view(batch_size ,num_q, -1)
        input_to_lstm = torch.cat((mlp_code_output, mlp_text_output.unsqueeze(1)), dim=1).float()
        print(input_to_lstm.shape)
        packed_output, (h_n, c_n) = self.lstm(input_to_lstm)
        output = self.layer1(h_n[-1])
        return output

##### Only Last Submission

In [14]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size_lstm=768, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm_snaphots = nn.LSTM(self.text_model.config.hidden_size, hidden_size_lstm, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_size_lstm + 1)
        self.lstm_submissions = nn.LSTM(hidden_size_lstm + 1, hidden_size_lstm, batch_first=True)
        self.MLP_text = nn.Sequential(
            nn.Linear(768, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.MLP_code = nn.Sequential(
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.fully_connected_layer = nn.Linear(hidden_size_lstm, 1)
        
    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        batch_size, q_num, c_padding, len = code_embedding.size()
        # snapshots_lstm = pack_padded_sequence(code_embedding.view(batch_size * q_num, c_padding, len), lengths=code_num.view(batch_size * q_num).to('cpu'), batch_first=True, enforce_sorted=False)
        # packed_output, (snapshots_h, c_n) = self.lstm_snaphots(snapshots_lstm)
        input_lstm_submissions = self.batch_norm(torch.cat((code_embedding.squeeze(2), prev_struggling.float().unsqueeze(-1)), dim=-1).view(batch_size * q_num, -1)).view(batch_size, q_num, -1)
        packed_output, (final_submission_h, c_n) = self.lstm_submissions(input_lstm_submissions)
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        text_mlp = self.MLP_text(text_output)
        code_mlp = self.MLP_code(final_submission_h.squeeze(0))
        combine_rep = text_mlp + code_mlp
        output = self.fully_connected_layer(combine_rep)
        return output

##### Without Binary

In [13]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size_lstm=512, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm_snaphots = nn.LSTM(self.text_model.config.hidden_size, hidden_size_lstm, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_size_lstm)
        self.lstm_submissions = nn.LSTM(hidden_size_lstm, hidden_size_lstm, batch_first=True)
        self.MLP_text = nn.Sequential(
            nn.Linear(768, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.MLP_code = nn.Sequential(
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.fully_connected_layer = nn.Linear(hidden_size_lstm, 1)
        
    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        batch_size, q_num, c_padding, len = code_embedding.size()
        snapshots_lstm = pack_padded_sequence(code_embedding.view(batch_size * q_num, c_padding, len), lengths=code_num.view(batch_size * q_num).to('cpu'), batch_first=True, enforce_sorted=False)
        packed_output, (snapshots_h, c_n) = self.lstm_snaphots(snapshots_lstm)
        # input_lstm_submissions = self.batch_norm(torch.cat((snapshots_h.view(batch_size, q_num, -1), prev_struggling.float().unsqueeze(-1)), dim=-1).view(batch_size * q_num, -1)).view(batch_size, q_num, -1)
        packed_output, (final_submission_h, c_n) = self.lstm_submissions(snapshots_h.view(batch_size, q_num, -1))
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        text_mlp = self.MLP_text(text_output)
        code_mlp = self.MLP_code(final_submission_h.squeeze(0))
        combine_rep = text_mlp + code_mlp
        output = self.fully_connected_layer(combine_rep)
        return output

##### ALL

In [25]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size_lstm=512, question_num=30):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm_snaphots = nn.LSTM(self.text_model.config.hidden_size, hidden_size_lstm, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_size_lstm + 1)
        self.lstm_submissions = nn.LSTM(hidden_size_lstm + 1, hidden_size_lstm, batch_first=True)
        self.MLP_text = nn.Sequential(
            nn.Linear(768, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.MLP_code = nn.Sequential(
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.fully_connected_layer = nn.Linear(hidden_size_lstm, 1)
        

    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num): 
        batch_size, q_num, c_padding, len = code_embedding.size()
        snapshots_lstm = pack_padded_sequence(code_embedding.view(batch_size * q_num, c_padding, len), lengths=code_num.view(batch_size * q_num).to('cpu'), batch_first=True, enforce_sorted=False)
        packed_output, (snapshots_h, c_n) = self.lstm_snaphots(snapshots_lstm)
        input_lstm_submissions = self.batch_norm(torch.cat((snapshots_h.view(batch_size, q_num, -1), prev_struggling.float().unsqueeze(-1)), dim=-1).view(batch_size * q_num, -1)).view(batch_size, q_num, -1)
        packed_output, (final_submission_h, c_n) = self.lstm_submissions(input_lstm_submissions)
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        text_mlp = self.MLP_text(text_output)
        code_mlp = self.MLP_code(final_submission_h.squeeze(0))
        combine_rep = text_mlp + code_mlp
        output = self.fully_connected_layer(combine_rep)
        return output

##### All with Taxonomy

In [ ]:
class SnapshotsClassifier(nn.Module):
    def __init__(self, text_model_name, hidden_size_lstm=512, question_num=30, num_feature=10):
        super(SnapshotsClassifier, self).__init__()
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.lstm_snaphots = nn.LSTM(self.text_model.config.hidden_size, hidden_size_lstm, batch_first=True)
        self.batch_norm_code_lstm = nn.BatchNorm1d(hidden_size_lstm + 1)
        self.batch_norm_text_mlp = nn.BatchNorm1d(self.text_model.config.hidden_size + num_feature)
        self.lstm_submissions = nn.LSTM(hidden_size_lstm + 1, hidden_size_lstm, batch_first=True)
        self.MLP_text = nn.Sequential(
            nn.Linear(self.text_model.config.hidden_size + num_feature, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.MLP_code = nn.Sequential(
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm),
            nn.ReLU(),
            nn.Linear(hidden_size_lstm, hidden_size_lstm),
            nn.LayerNorm(hidden_size_lstm)
        )
        self.fully_connected_layer = nn.Linear(hidden_size_lstm, 1)
        
    def forward(self, text_input_ids, text_attention_mask, code_embedding, code_num, questions_input_ids, questions_attention_mask, prev_struggling, que_num, features): 
        batch_size, q_num, c_padding, len = code_embedding.size()
        snapshots_lstm = pack_padded_sequence(code_embedding.view(batch_size * q_num, c_padding, len), lengths=code_num.view(batch_size * q_num).to('cpu'), batch_first=True, enforce_sorted=False)
        packed_output, (snapshots_h, c_n) = self.lstm_snaphots(snapshots_lstm)
        input_lstm_submissions = self.batch_norm_code_lstm(torch.cat((snapshots_h.view(batch_size, q_num, -1), prev_struggling.float().unsqueeze(-1)), dim=-1).view(batch_size * q_num, -1)).view(batch_size, q_num, -1)
        packed_output, (final_submission_h, c_n) = self.lstm_submissions(input_lstm_submissions)
        with torch.no_grad():
            text_output = self.text_model(text_input_ids, text_attention_mask).last_hidden_state[:, 0, :]
        input_text_mlp = self.batch_norm_text_mlp(torch.cat((features, text_output), dim=-1))
        text_mlp = self.MLP_text(input_text_mlp)
        code_mlp = self.MLP_code(final_submission_h.squeeze(0))
        combine_rep = text_mlp + code_mlp
        output = self.fully_connected_layer(combine_rep)
        return output

# More

## Falcon

In [ ]:
path = "/home/nogaschw/Codeworkout/df_taxonomy_falcon_new_label.pkl"
df = pd.read_pickle(path)

df.fillna(0, inplace=True)

def arrange(df1, st_point=0, end_point=30):
    df1 = df1[df1['prev_question'].apply(lambda x: len(x) > end_point and len(x) <= end_point + 20)]
    df1['prev_code'] = df1['prev_code'].apply(lambda x: x[st_point:end_point])
    df1['prev_question'] = df1['prev_question'].apply(lambda x: x[st_point:end_point])
    df1['prev_label'] = df1['prev_label'].apply(lambda x: x[st_point:end_point])
    df1 = df1[df1['student_id'].map(df1['student_id'].value_counts()) == 20]
    return df1

df_1 = arrange(df)
df_2 = arrange(df, 50, 80) # More data
df = pd.concat([df_1, df_2], ignore_index=True)

# df = df[df.type == 'lab'] # Only Lab

## Codeworkout

In [ ]:
path = "/home/nogaschw/Codeworkout/df_codeworkout_TL_Tax.pkl"
df = pd.read_pickle(path)

df['IfElse'] = df[['If/Else', 'NestedIf']].max(axis=1)
df['Loops'] = df[['While', 'For', 'NestedFor']].max(axis=1)
df['MathOperations'] = df[['Math+-*/', 'Math%']].max(axis=1)
df['LogicOperators'] = df[['LogicAndNotOr', 'LogicCompareNum', 'LogicBoolean']].max(axis=1)
df['StringOperations'] = df[["StringFormat", "StringConcat", "StringIndex", "StringLen", "StringEqual", "CharEqual"]].max(axis=1)
df["List"]= df["ArrayIndex"]
df['FileOperations'] = 0
df["Functions"] = df["DefFunction"]
df['Dictionary'] = 0
df['Tuple'] = 0

df = df.drop(columns=['If/Else', 'NestedIf', 'StringConcat', 'StringIndex', 'StringFormat',"StringLen", "StringEqual", 'ArrayIndex', 'DefFunction',
                      'While', 'For', 'NestedFor', 'Math+-*/', 'Math%', 'LogicAndNotOr', 'LogicCompareNum', 'LogicBoolean', 'CharEqual'])


df.fillna(0, inplace=True)
df['prev_code'] = df['prev_code'].apply(lambda x: [list(set(i)) for i in x])


In [ ]:
# New use in taxonamy
skills = pd.DataFrame({
    'problem_id': [41, 43, 44, 45, 46, 48, 49, 51, 56, 57, 64, 67, 70, 71, 104, 106, 107, 108, 112, 118],
    'comprehension':  [0.1, 0.2, 0.3, 0.9, 0.8, 0.5, 1.0, 0.7, 0.6, 0.9, 0.7, 0.4, 0.8, 0.6, 0.2, 0.5, 0.3, 0.6, 0.8, 0.1] ,
    'alg_design':  [0.6, 0.7, 0.8, 0.9, 0.4, 1.0, 0.5, 0.3, 0.1, 0.2, 0.8, 0.7, 0.6, 0.5, 0.3, 0.1, 1.0, 0.9, 0.4, 0.2] ,
    'decomposition':  [0.1, 0.2, 0.3, 0.7, 0.7, 0.4, 1.0, 0.8, 0.5, 0.9, 0.8, 0.5, 0.7, 0.6, 0.2, 0.6, 0.3, 0.9, 0.8, 0.3]
})
df = df.merge(skills, on='problem_id')